# Introduction

Récupération de plongements de mots contextuels à partir de BERT.

Adapté d'un notebook de Chris McCormick.


# 1. Charger un modèle BERT

In [1]:
!pip install transformers

  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/a2/f6/587c62fd21fc988555b85351f50bbde43a51524caafd63bc69240ded14fd/sentencepiece-0.2.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --- ------------------------------------ 92.2/991.5 kB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 184.3/991.5 kB 2.2 MB/s eta 0:00:01
   ---------- ----------------------------- 256.0/991.5 kB 2.0 MB/s eta 0:00:01
   -------------- ------------------------- 368.6/991.5 kB 2.1 MB/s eta 0:00:01
   ------------------ --------------------- 460.8/991.5 kB 1.9 MB/s eta 0:00:01
   ----------------------- ---------------- 583.7/991.5 kB 2.0 MB/s eta 0:00:01
   --------------------------- ------------ 675.8/991.5 kB 2.0 MB/s eta 0:00:01
   ------------------------------- -------- 778.2/991.5 kB 2.0 MB/s eta 0:00:01
   -------------------------------- ------- 809.0/991.5 kB 2.0 MB/s et

In [5]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


100%|██████████████████████████████████████████████████████████████████████| 231508/231508 [00:00<00:00, 1854005.39B/s]


# 2. Formatage de l'entrée
Étant donné que BERT est un modèle pré-entraîné qui attend des données d'entrée dans un format spécifique, nous aurons besoin de :

1. Un **jeton spécial, `[SEP]`,** pour marquer la fin d'une phrase ou la séparation entre deux phrases
2. Un **jeton spécial, `[CLS]`,** au début de notre texte. Ce jeton est utilisé pour les tâches de classification, mais BERT l'attend quelle que soit votre application.
3. Jetons conformes au vocabulaire fixe utilisé dans BERT
4. Les **ID de jeton** pour les jetons, du tokenizer de BERT
5. **Identifiants de masque** pour indiquer quels éléments de la séquence sont des jetons et lesquels sont des éléments de remplissage
6. **IDs de segment** utilisés pour distinguer différentes phrases
7. **Embeddings positionnels** utilisés pour afficher la position du jeton dans la séquence

Heureusement, l'interface `transformers` prend en charge toutes les exigences ci-dessus (en utilisant la fonction `tokenizer.encode_plus`).

Puisqu'il s'agit d'une introduction au travail avec BERT, nous allons effectuer ces étapes de manière (principalement) manuelle.

## 2.1. Jetons spéciaux
BERT peut prendre en entrée une ou deux phrases et utilise le jeton spécial `[SEP]` pour les différencier. Le jeton `[CLS]` apparaît toujours en début de texte, et est spécifique aux tâches de classification.

Cependant, les deux jetons sont *toujours requis*, même si nous n'avons qu'une seule phrase et même si nous n'utilisons pas BERT pour la classification. C'est ainsi que BERT a été pré-entrainé, et c'est donc ce que BERT s'attend à voir.

**Saisie de 2 phrases** :

`[CLS] The man went to the store. [SEP] He bought a gallon of milk.`

**1 phrase en entrée **:

`[CLS] The man went to the store. [SEP]`



## 2.2. Segmentation

BERT fournit son propre tokenizer, que nous avons importé ci-dessus. 

In [6]:
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]']


Voici quelques exemples de jetons contenus dans notre vocabulaire. Les jetons commençant par deux ## sont des sous-mots ou des caractères individuels.

In [7]:
list(tokenizer.vocab.keys())[5000:5020]

['knight',
 'lap',
 'survey',
 'ma',
 '##ow',
 'noise',
 'billy',
 '##ium',
 'shooting',
 'guide',
 'bedroom',
 'priest',
 'resistance',
 'motor',
 'homes',
 'sounded',
 'giant',
 '##mer',
 '150',
 'scenes']

Après avoir divisé le texte en jetons, nous devons ensuite convertir la phrase d'une liste de chaînes en une liste d'indices de vocabulaire.

À partir de maintenant, nous utiliserons l’exemple de phrase ci-dessous, qui contient deux instances du mot « banque » avec des significations différentes.

In [8]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
after         2,044
stealing     11,065
money         2,769
from          2,013
the           1,996
bank          2,924
vault        11,632
,             1,010
the           1,996
bank          2,924
robber       27,307
was           2,001
seen          2,464
fishing       5,645
on            2,006
the           1,996
mississippi   5,900
river         2,314
bank          2,924
.             1,012
[SEP]           102


## 2.3. Segment ID
BERT est formé et attend des paires de phrases, en utilisant des 1 et des 0 pour distinguer les deux phrases. Autrement dit, pour chaque jeton de « tokenized_text », nous devons spécifier à quelle phrase il appartient : phrase 0 (une série de 0) ou phrase 1 (une série de 1). Pour nos besoins, les entrées d'une seule phrase ne nécessitent qu'une série de 1, nous allons donc créer un vecteur de 1 pour chaque jeton de notre phrase d'entrée.

Si vous souhaitez traiter deux phrases, attribuez à chaque mot de la première phrase plus le jeton « [SEP] » un 0, et à tous les jetons de la deuxième phrase un 1.

In [9]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# 3. Extraction de plongements



## 3.1. Exécuter BERT sur notre texte



Nous devons convertir nos données en tenseurs Torch et appeler le modèle BERT. L'interface BERT PyTorch nécessite que les données soient dans ces tenseurs  plutôt que dans des listes Python, nous convertissons donc les listes ici


In [10]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

L'appel de `from_pretrained` récupérera le modèle sur Internet. Lorsque nous chargeons le `bert-base-uncased`, nous voyons la définition du modèle imprimée dans la journalisation. 

model.eval() met notre modèle en mode évaluation plutôt qu'en mode entrainement. Dans ce cas, le mode d'évaluation désactive la régularisation dropout qui est utilisée dans l'entrainement.

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


 71%|█████████████████████████████████████████████▌                  | 313361408/440473133 [02:29<01:00, 2114016.92B/s]

évaluons BERT sur notre exemple de texte et récupérons les états cachés du réseau 

In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers.
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on
    # how it's  configured in the `from_pretrained` call earlier. In this case,
    # becase we set `output_hidden_states = True`, the third item will be the
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

## 3.2. Comprendre la sortie


Nous voyons l'ensemble complet des états cachés de ce modèle, stockés dans l'objet `hidden_states`. Cet objet a quatre dimensions, dans l'ordre suivant :

1. Le numéro de couche (13 couches)
2. Le numéro de lot (1 phrase)
3. Le numéro de mot/jeton (22 jetons dans notre phrase)
4. L'unité cachée (768 dimensions)

La premiere couche est constituée des plongements en entrée, le reste est constitué des sorties de chacune des 12 couches de BERT.

Cela représente 219 648 valeurs uniques rien que pour représenter notre seule phrase !

La deuxième dimension, la taille du lot, est utilisée lors de la soumission simultanée de plusieurs phrases au modèle ; ici, cependant, nous n’avons qu’un exemple de phrase.

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 22
Number of hidden units: 768


## 3.3. Création de vecteurs de mots et de phrases à partir d'états cachés

Nous aimerions obtenir des vecteurs individuels pour chacun de nos jetons, ou peut-être une seule représentation vectorielle de la phrase entière, mais pour chaque jeton de notre entrée, nous avons 13 vecteurs distincts chacun d'une longueur de 768.

Afin d'obtenir les vecteurs individuels, nous devrons combiner certains des vecteurs de couches... mais quelle couche ou combinaison de couches offre la meilleure représentation ?

Malheureusement, il n’existe pas de réponse simple. Essayons cependant quelques approches raisonnables. 



### Plongements de mots

Pour vous donner quelques exemples, créons des vecteurs de mots de deux manières.

Tout d'abord, **concaténons** les quatre dernières couches, ce qui nous donne un seul vecteur de mot par jeton. Chaque vecteur aura une longueur de « 4 x 768 = 3 072 ».

In [ ]:
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 22 x 3072



Comme méthode alternative, essayons de créer les vecteurs de mots en **sommant** les quatre dernières couches.

In [ ]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)

    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 22 x 768


### Plongements de phrases


Pour obtenir un seul vecteur pour l'ensemble de notre phrase, nous avons plusieurs stratégies dépendant de l'application, mais une approche simple consiste à faire la moyenne de la deuxième à la dernière couche cachée de chaque jeton, produisant un seul vecteur de longueur 768.

Il convient de noter que bien que le [CLS] agisse comme une « représentation agrégée » pour les tâches de classification, ce n'est pas le meilleur choix pour un plongement de phrases. 

In [ ]:
# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


## 3.4. Test des vecteurs contextuellement dépendants
Pour confirmer que la valeur de ces vecteurs dépend en fait du contexte, regardons les différentes instances du mot « banque » dans notre exemple de phrase :

"After stealing money from the **bank vault**, the **bank robber** was seen fishing on the Mississippi **river bank**."

Trouvons l'index de ces trois instances du mot « banque » dans la phrase d'exemple.

In [ ]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 after
2 stealing
3 money
4 from
5 the
6 bank
7 vault
8 ,
9 the
10 bank
11 robber
12 was
13 seen
14 fishing
15 on
16 the
17 mississippi
18 river
19 bank
20 .
21 [SEP]



Ils sont à 6, 10 et 19.

Pour cette analyse, nous utiliserons les vecteurs de mots que nous avons créés en additionnant les quatre dernières couches.

Nous pouvons essayer d'imprimer leurs vecteurs pour les comparer.

In [ ]:
print('First 5 vector values for each instance of "bank".')
print('')
print("bank vault   ", str(token_vecs_sum[6][:5]))
print("bank robber  ", str(token_vecs_sum[10][:5]))
print("river bank   ", str(token_vecs_sum[19][:5]))

First 5 vector values for each instance of "bank".

bank vault    tensor([ 3.3596, -2.9805, -1.5421,  0.7065,  2.0031])
bank robber   tensor([ 2.7359, -2.5577, -1.3094,  0.6797,  1.6633])
river bank    tensor([ 1.5266, -0.8895, -0.5152, -0.9298,  2.8334])


Nous pouvons voir que les valeurs diffèrent, mais calculons la similarité cosinus entre les vecteurs pour faire une comparaison plus précise.

In [ ]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for  *similar*  meanings:  0.94
Vector similarity for *different* meanings:  0.69


## 3.5. Stratégie de pooling et choix des couches

(Image de [Jay Allamar](http://jalammar.github.io/illustrated-bert/))


![alt text](http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png)


**BERT-as-service**

Vous pouvez aussi consulter [bert-as-service](https://github.com/hanxiao/bert-as-service) 
